# Imports

In [2]:
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
import optuna
from joblib import Parallel, delayed
from utils.Loader import NEUDataset
from utils.Perspectiver import Perspectiver
from source.Prototype1 import Prototype1
from source.F import Autoencoder

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import multiprocessing
import torch
from mpl_toolkits.mplot3d import Axes3D
from skimage.restoration import denoise_wavelet
import random
import math
from PIL import Image
from collections import deque
import numpy as np
from scipy.ndimage import maximum_filter, minimum_filter, label, generate_binary_structure
from concurrent.futures import ProcessPoolExecutor, as_completed
from scipy.ndimage import label as ndi_label, binary_dilation

# Init variables

In [3]:
class DenormalizeToUint8(object):
    """
    Transform que desnormaliza una imagen tensor (si se proporcionan mean y std) y la convierte a uint8 en [0, 255].
    """
    def __init__(self, mean=None, std=None):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        # Desnormaliza cada canal si se especificaron media y desviación
        if self.mean is not None and self.std is not None:
            # Se asume tensor de forma (C, H, W)
            for t, m, s in zip(tensor, self.mean, self.std):
                t.mul_(s).add_(m)
        # Convierte de [0,1] a [0,255] y a tipo uint8
        return (tensor * 255).clamp(0, 255).byte()

    def __repr__(self):
        return f"{self.__class__.__name__}(mean={self.mean}, std={self.std})"

In [4]:
autoencoder = Autoencoder()
ground_truth = NEUDataset(set="clustered/train", seed=1234, scale=1, transform=DenormalizeToUint8())
train_images = NEUDataset(set="train", seed=1234, scale=0.5, transform=DenormalizeToUint8())

image, _ = ground_truth.__getitem__(index=1)
image.shape
image

Dataset: clustered/train created!
Dataset: train created!


tensor([[[0.5451, 0.5451, 0.5412,  ..., 0.4745, 0.4745, 0.4784],
         [0.5412, 0.5412, 0.5373,  ..., 0.4745, 0.4784, 0.4784],
         [0.5255, 0.5294, 0.5333,  ..., 0.4784, 0.4824, 0.4824],
         ...,
         [0.4745, 0.4745, 0.4745,  ..., 0.4627, 0.4627, 0.4627],
         [0.4745, 0.4745, 0.4745,  ..., 0.4627, 0.4627, 0.4627],
         [0.4745, 0.4745, 0.4745,  ..., 0.4627, 0.4627, 0.4627]]])

# Hyperparameters

In [5]:
LEARNING_RATE = 0.00001
EPOCH = 1000
BATCH_SIZE = len(ground_truth)//2
THREADS = 8

# Training

In [6]:
ground_truth_loader = DataLoader(dataset=ground_truth, batch_size=BATCH_SIZE, shuffle=True, num_workers=THREADS) # __getitem__ return image, _ (useless)
train_images_loader = DataLoader(dataset=train_images, batch_size=BATCH_SIZE, shuffle=True, num_workers=THREADS)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

autoencoder.to(device)
criterion = nn.MSELoss(reduction="sum")  # Compara la imagen original con la reconstruida
optimizer = Adam(autoencoder.parameters(), lr=LEARNING_RATE)

In [7]:
autoencoder.train()  # Modo entrenamiento

for epoch in range(EPOCH):
    running_loss = 0.0
    # Iteramos simultáneamente sobre los DataLoaders
    for (inputs, _), (targets, _) in zip(train_images_loader, ground_truth_loader):
        inputs = (inputs).to(device)
        targets = (targets).to(device)
        #print(inputs.shape)
        #print(targets.shape)
        optimizer.zero_grad()         # Reinicia los gradientes
        outputs = autoencoder(inputs)         # Propagación hacia adelante
        loss = criterion(outputs, inputs)  # Cálculo de la pérdida
        loss.backward()               # Retropropagación
        optimizer.step()              # Actualización de parámetros
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_images_loader.dataset)
    print(f"Epoch [{epoch+1}/{EPOCH}], Loss: {epoch_loss:.4f}")

Epoch [1/1000], Loss: 280045.0968
Epoch [2/1000], Loss: 281222.0784
Epoch [3/1000], Loss: 279646.5131
Epoch [4/1000], Loss: 278074.6704
Epoch [5/1000], Loss: 280595.6643
Epoch [6/1000], Loss: 278536.0340
Epoch [7/1000], Loss: 281012.6357
Epoch [8/1000], Loss: 279682.2963
Epoch [9/1000], Loss: 280610.6760
Epoch [10/1000], Loss: 277944.1031
Epoch [11/1000], Loss: 279784.0533
Epoch [12/1000], Loss: 279451.9094
Epoch [13/1000], Loss: 279537.3777
Epoch [14/1000], Loss: 280830.2536
Epoch [15/1000], Loss: 280371.3439
Epoch [16/1000], Loss: 278741.0796
Epoch [17/1000], Loss: 279861.9955
Epoch [18/1000], Loss: 280070.6722
Epoch [19/1000], Loss: 279933.9370
Epoch [20/1000], Loss: 278701.5942
Epoch [21/1000], Loss: 279373.1612
Epoch [22/1000], Loss: 280965.9880
Epoch [23/1000], Loss: 279525.6526
Epoch [24/1000], Loss: 281954.7190
Epoch [25/1000], Loss: 280173.0419
Epoch [26/1000], Loss: 278713.4805
Epoch [27/1000], Loss: 280142.1863
Epoch [28/1000], Loss: 281533.8644
Epoch [29/1000], Loss: 280020

KeyboardInterrupt: 